### GreenTrace Demo

This is an example of a series of events, pushed to the events pool by an entity which participates in the GreenTrace Project, waiting to be mined

In [27]:
events =  {
        "event1": {
            "type": "new_material",
            "material_id": "uuid-material-1234",
            "material_reference": "http://example.com/materials/aluminum-cans-info"
        },
        "event2": {
            "type": "new_entity",
            "entity_id": "uuid-entity-5678",
            "entity_reference": "Aluminum Recycling Co., 123 Recycle Rd., Eco City"
        },
        "event3": {
            "type": "new_process",
            "process_id": "uuid-process-9012",
            "process_reference": "Transforming aluminum cans into aluminum sheets"
        },
        "event4": {
            "type": "checkpoint",
            "checkpoint_id": "uuid-checkpoint-3456",
            "original_product_upc": "012345678912",
            "quantity": 5000,
            "item_ref": None,
            "process_ids": ["uuid-process-9012"],
            "material_out_id": "uuid-material-out-7890",
            "origin_checkpoint": "uuid-origin-1122",
            "destination_checkpoint": "uuid-destination-3344",
            "entity_id": "uuid-entity-5678",
            "timestamp_in": "2023-11-29T10:00:00Z",
            "timestamp_out": "2023-11-29T16:00:00Z"
        }
}

In [28]:
from fastapi.testclient import TestClient

from app import app
import json

client = TestClient(app)

In [29]:
response = client.post("/events", json={
	"events": events
})

response = client.get("/transactions")
response.json()["transactions"]

{}

We store our chain in the chain.json file. The API represents the consensus of the network. And a miner (GreenTrace Vendor) gets the chain, adds a block, and send back a new chain which the API will validate.

In [30]:
client.get("/chain").json()

{'chain': [{'id': 0,
   'previous_hash': '0',
   'nonce': 67170,
   'timestamp': 1699427804.6617067,
   'data': ['Genesis block.'],
   'hash': '000005163dbcbd434f5c0c055a2a874a4f2e7c3dcd3f4df8dad07e80b13445bc'}]}

In [ ]:
from chain import Blockchain
from miner import Miner

chain = Blockchain("chain.json")

miner=Miner(wallet_id="12345")

The miner now adds a new block to the chain, with 5 leading zeros at its hash

In [33]:

chain = Blockchain("chain.json")
miner.mine(transaction_pool=events, blockchain=chain, difficulty=5)
json={
	"chain": chain.as_dict_list()
}


response = client.post("/chain", json=json)
response

<Response [200 OK]>

Network accepts new GreenTrace chain

In [34]:
client.get("/chain").json()

{'chain': [{'id': 0,
   'previous_hash': '0',
   'nonce': 67170,
   'timestamp': 1699427804.6617067,
   'data': ['Genesis block.'],
   'hash': '000005163dbcbd434f5c0c055a2a874a4f2e7c3dcd3f4df8dad07e80b13445bc'},
  {'id': 1,
   'previous_hash': '000005163dbcbd434f5c0c055a2a874a4f2e7c3dcd3f4df8dad07e80b13445bc',
   'nonce': 1632558,
   'timestamp': 1701276541.1848588,
   'data': {'event1': {'type': 'new_material',
     'material_id': 'uuid-material-1234',
     'material_reference': 'http://example.com/materials/aluminum-cans-info'},
    'event2': {'type': 'new_entity',
     'entity_id': 'uuid-entity-5678',
     'entity_reference': 'Aluminum Recycling Co., 123 Recycle Rd., Eco City'},
    'event3': {'type': 'new_process',
     'process_id': 'uuid-process-9012',
     'process_reference': 'Transforming aluminum cans into aluminum sheets'},
    'event4': {'type': 'checkpoint',
     'checkpoint_id': 'uuid-checkpoint-3456',
     'original_product_upc': '012345678912',
     'quantity': 5000,
  